# 리소스 정리
이 노트북에서는 저장된 변수 이름을 로드하여 이전 노트북에서 만든 모든 리소스를 정리하는 방법을 보여줍니다.

(!) 여러 노트북을 사용하여 리소스를 만들었고 여러 데이터 세트 그룹이 있는 경우, 노트북 끝에서 `# store for cleanup`으로 시작하는 셀을 실행한 다음 그 직후에 각 리소스에 대해 이 노트북을 실행합니다(데이터 세트 그룹을 2개 만든 경우 이 노트북을 두 번 실행해야 함). 이 코드는 `# store for cleanup`에 지정된 리소스만 삭제하며, 여러 노트북을 사용한 경우 마지막으로 실행한 노트북에 해당하는 값이 여기에 저장됩니다.

이 노트북은 아래에 정의된 함수를 사용하여 데이터 세트 그룹 내의 리소스마다 반복합니다.

이 노트북을 사용하여 Amazon Personalize 리소스를 삭제할 수 있습니다. 리소스 ARN은 이전 노트북에 정의되어 있거나, 아니면 아래에 수동으로 입력할 수 있습니다.

In [ ]:
%store -r

In [ ]:
# If you cannot/do not want to use "%store -r" to load the resources to delete, 
# you can uncomment the code bellow and enter them manually

# dataset_group_arn='XXXXX'
# role_name='XXXXX'
# region='XXXXX'

삭제할 리소스를 인쇄합니다. 삭제하려는 리소스에 해당하는지 확인하세요.
이 작업은 취소할 수 없습니다.

In [ ]:
print ('dataset_group_arn:', dataset_group_arn)
print ('role_name:', role_name)
print ('region:', region)

In [ ]:
schema_arns = []

In [ ]:
import sys
import getopt
import logging
import botocore
import boto3
import time
from packaging import version
from time import sleep
from botocore.exceptions import ClientError

logger = logging.getLogger()
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)
logger.setLevel(logging.INFO)
logger.addHandler(handler)

personalize = None

In [ ]:
def _delete_event_trackers(dataset_group_arn):
    event_tracker_arns = []

    event_trackers_paginator = personalize.get_paginator('list_event_trackers')
    for event_tracker_page in event_trackers_paginator.paginate(datasetGroupArn = dataset_group_arn):
        for event_tracker in event_tracker_page['eventTrackers']:
            if event_tracker['status'] in [ 'ACTIVE', 'CREATE FAILED' ]:
                logger.info('Deleting event tracker {}'.format(event_tracker['eventTrackerArn']))
                personalize.delete_event_tracker(eventTrackerArn = event_tracker['eventTrackerArn'])
            elif event_tracker['status'].startswith('DELETE'):
                logger.warning('Event tracker {} is already being deleted so will wait for delete to complete'.format(event_tracker['eventTrackerArn']))
            else:
                raise Exception('Solution {} has a status of {} so cannot be deleted'.format(event_tracker['eventTrackerArn'], event_tracker['status']))

            event_tracker_arns.append(event_tracker['eventTrackerArn'])

    max_time = time.time() + 30*60 # 30 mins
    while time.time() < max_time:
        for event_tracker_arn in event_tracker_arns:
            try:
                describe_response = personalize.describe_event_tracker(eventTrackerArn = event_tracker_arn)
                logger.debug('Event tracker {} status is {}'.format(event_tracker_arn, describe_response['eventTracker']['status']))
            except ClientError as e:
                error_code = e.response['Error']['Code']
                if error_code == 'ResourceNotFoundException':
                    event_tracker_arns.remove(event_tracker_arn)

        if len(event_tracker_arns) == 0:
            logger.info('All event trackers have been deleted or none exist for dataset group')
            break
        else:
            logger.info('Waiting for {} event tracker(s) to be deleted'.format(len(event_tracker_arns)))
            time.sleep(20)

    if len(event_tracker_arns) > 0:
        raise Exception('Timed out waiting for all event trackers to be deleted')

def _delete_filters(dataset_group_arn):
    filter_arns = []

    filters_response = personalize.list_filters(datasetGroupArn = dataset_group_arn, maxResults = 100)
    for filter in filters_response['Filters']:
        logger.info('Deleting filter ' + filter['filterArn'])
        personalize.delete_filter(filterArn = filter['filterArn'])
        filter_arns.append(filter['filterArn'])

    max_time = time.time() + 30*60 # 30 mins
    while time.time() < max_time:
        for filter_arn in filter_arns:
            try:
                describe_response = personalize.describe_filter(filterArn = filter_arn)
                logger.debug('Filter {} status is {}'.format(filter_arn, describe_response['filter']['status']))
            except ClientError as e:
                error_code = e.response['Error']['Code']
                if error_code == 'ResourceNotFoundException':
                    filter_arns.remove(filter_arn)

        if len(filter_arns) == 0:
            logger.info('All filters have been deleted or none exist for dataset group')
            break
        else:
            logger.info('Waiting for {} filter(s) to be deleted'.format(len(filter_arns)))
            time.sleep(20)

    if len(filter_arns) > 0:
        raise Exception('Timed out waiting for all filter(s) to be deleted')
        
def _delete_recommenders(dataset_group_arn):
    recommender_arns = []
    recommenders_response = personalize.list_recommenders(datasetGroupArn = dataset_group_arn, maxResults = 100)
    for recommender in recommenders_response['recommenders']:
        logger.info('Deleting recommender ' + recommender['recommenderArn'])
        personalize.delete_recommender(recommenderArn = recommender['recommenderArn'])
        recommender_arns.append(recommender['recommenderArn'])
    max_time = time.time() + 30*60 # 30 mins
    while time.time() < max_time:
        for recommender_arn in recommender_arns:
            try:
                describe_response = personalize.describe_recommender(recommenderArn = recommender_arn)
                logger.debug('Recommender {} status is {}'.format(recommender_arn, describe_response['recommender']['status']))
            except ClientError as e:
                error_code = e.response['Error']['Code']
                if error_code == 'ResourceNotFoundException':
                    recommender_arns.remove(recommender_arn)

        if len(recommender_arns) == 0:
            logger.info('All recommenders have been deleted or none exist for dataset group')
            break
        else:
            logger.info('Waiting for {} recommender(s) to be deleted'.format(len(recommender_arns)))
            time.sleep(20)

    if len(recommender_arns) > 0:
        raise Exception('Timed out waiting for all recommender(s) to be deleted')
    

def _delete_datasets_and_schemas(dataset_group_arn, schema_arns):
    dataset_arns = []
    
    dataset_paginator = personalize.get_paginator('list_datasets')
    for dataset_page in dataset_paginator.paginate(datasetGroupArn = dataset_group_arn):
        for dataset in dataset_page['datasets']:
            describe_response = personalize.describe_dataset(datasetArn = dataset['datasetArn'])
            schema_arns.append(describe_response['dataset']['schemaArn'])

            if dataset['status'] in ['ACTIVE', 'CREATE FAILED']:
                logger.info('Deleting dataset ' + dataset['datasetArn'])
                personalize.delete_dataset(datasetArn = dataset['datasetArn'])
            elif dataset['status'].startswith('DELETE'):
                logger.warning('Dataset {} is already being deleted so will wait for delete to complete'.format(dataset['datasetArn']))
            else:
                raise Exception('Dataset {} has a status of {} so cannot be deleted'.format(dataset['datasetArn'], dataset['status']))

            dataset_arns.append(dataset['datasetArn'])

    max_time = time.time() + 30*60 # 30 mins
    while time.time() < max_time:
        for dataset_arn in dataset_arns:
            try:
                describe_response = personalize.describe_dataset(datasetArn = dataset_arn)
                logger.debug('Dataset {} status is {}'.format(dataset_arn, describe_response['dataset']['status']))
            except ClientError as e:
                error_code = e.response['Error']['Code']
                if error_code == 'ResourceNotFoundException':
                    dataset_arns.remove(dataset_arn)

        if len(dataset_arns) == 0:
            logger.info('All datasets have been deleted or none exist for dataset group')
            break
        else:
            logger.info('Waiting for {} dataset(s) to be deleted'.format(len(dataset_arns)))
            time.sleep(20)

    if len(dataset_arns) > 0:
        raise Exception('Timed out waiting for all datasets to be deleted')

    for schema_arn in schema_arns:
        try:
            logger.info('Deleting schema ' + schema_arn)
            personalize.delete_schema(schemaArn = schema_arn)
        except ClientError as e:
            error_code = e.response['Error']['Code']
            if error_code == 'ResourceInUseException':
                logger.info('Schema {} is still in-use by another dataset (likely in another dataset group)'.format(schema_arn))
            else:
                raise e

    logger.info('All schemas used exclusively by datasets have been deleted or none exist for dataset group')

def _delete_dataset_group(dataset_group_arn):
    logger.info('Deleting dataset group ' + dataset_group_arn)
    personalize.delete_dataset_group(datasetGroupArn = dataset_group_arn)

    max_time = time.time() + 30*60 # 30 mins
    while time.time() < max_time:
        try:
            describe_response = personalize.describe_dataset_group(datasetGroupArn = dataset_group_arn)
            logger.debug('Dataset group {} status is {}'.format(dataset_group_arn, describe_response['datasetGroup']['status']))
            break
        except ClientError as e:
            error_code = e.response['Error']['Code']
            if error_code == 'ResourceNotFoundException':
                logger.info('Dataset group {} has been fully deleted'.format(dataset_group_arn))
            else:
                raise e

        logger.info('Waiting for dataset group to be deleted')
        time.sleep(20)

def delete_dataset_groups(dataset_group_arns, schema_arns, region = None):
    global personalize
    personalize = boto3.client(service_name = 'personalize', region_name = region)

    for dataset_group_arn in dataset_group_arns:
        logger.info('Dataset Group ARN: ' + dataset_group_arn)

        # 1. Delete Recommenders
        _delete_recommenders(dataset_group_arn)
        
        # 2. Delete event trackers
        _delete_event_trackers(dataset_group_arn)

        # 3. Delete filters
        _delete_filters(dataset_group_arn)

        # 4. Delete datasets and their schemas
        _delete_datasets_and_schemas(dataset_group_arn, schema_arns)

        # 5. Delete dataset group
        _delete_dataset_group(dataset_group_arn)

        logger.info(f'Dataset group {dataset_group_arn} fully deleted')


In [ ]:
delete_dataset_groups([dataset_group_arn], schema_arns, region)

## IAM 역할 정리
역할을 삭제하는 것부터 시작합니다.

In [ ]:
iam = boto3.client('iam')

삭제할 역할의 이름을 식별합니다.

아직 정책이 연결되어 있는 IAM 역할은 삭제할 수 없습니다. 따라서 관련 역할을 식별한 후, 해당 역할에 연결된 정책을 나열해 봅니다.

In [ ]:
iam.list_attached_role_policies(
    RoleName = role_name
)

In [ ]:
# detaching the role policies
iam.detach_role_policy(
    RoleName = role_name,
    PolicyArn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
)
iam.detach_role_policy(
    RoleName = role_name,
    PolicyArn = 'arn:aws:iam::aws:policy/AmazonS3FullAccess'
)

이제 IAM 역할을 삭제할 수 있습니다.

In [ ]:
iam.delete_role(
    RoleName = role_name
)

## S3 버킷 삭제
S3 버킷을 삭제하려면 먼저 버킷이 비어 있어야 합니다. S3 버킷을 삭제하는 가장 쉬운 방법은 AWS 콘솔에서 S3으로 이동하여 버킷의 객체를 삭제한 다음 S3 버킷 자체를 삭제하는 것입니다.